In [2]:
from compas.datastructures import Mesh
from compas.datastructures import subdivision as sd
import ipyvolume as ipv
from utilities import draw_compas_mesh
from compas.geometry import Vector

In [3]:
from utils import whiteboard as wb

In [6]:
mesh = Mesh.from_polyhedron(12)
mesh2 = sd.mesh_subdivide_tri(mesh)
mesh3 = sd.trimesh_subdivide_loop(mesh2)

In [17]:
draw_compas_mesh(mesh3)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [8]:
wb()

In [10]:
subd = mesh3.copy()
height = 0.1
for fkey in mesh3.faces():
    centroid = mesh3.face_centroid(fkey)
    centroid_vector = Vector(*centroid)
    normal = mesh3.face_normal(fkey)
    normal_vector = Vector(*normal)
    new_vertex = centroid_vector + normal_vector * height
    subd.insert_vertex(fkey, xyz=new_vertex)

In [11]:
draw_compas_mesh(subd)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [12]:
def mesh_subdivide_pyramid(mesh, k=1, height=1.0):
    """
    Subdivide a mesh using insertion of a vertex at centroid + height * face normal.

    Parameters
    ----------
    mesh : Mesh
        The mesh object that will be subdivided.
    k : int
        Optional. The number of levels of subdivision. Default is ``1``.
    height : float
        The distance of the new vertex to the face.

    Returns
    -------
    Mesh
        A new subdivided mesh.
    """
    subd = mesh.copy()
    for fkey in mesh.faces():
        centroid = mesh.face_centroid(fkey)
        centroid_vector = Vector(*centroid)
        normal = mesh.face_normal(fkey)
        normal_vector = Vector(*normal)
        new_vertex = centroid_vector + normal_vector * height
        subd.insert_vertex(fkey, xyz=new_vertex)
    return subd

In [13]:
our_mesh = mesh_subdivide_pyramid(mesh3, height=0.3)
draw_compas_mesh(our_mesh)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [14]:
wb()

In [21]:
def mesh_subdivide_tapered(mesh, k=1, height=1.0, ratio=0.5):
    """
    TODO
    """
    subd = mesh.copy()
    for fkey in mesh.faces():
        centroid = mesh.face_centroid(fkey)
        centroid_vector = Vector(*centroid)
        normal = mesh.face_normal(fkey)
        normal_vector = Vector(*normal)
        normal_vector *= height
        
        face_verts = mesh.face_vertices(fkey)
        new_verts = []
        for v in face_verts:
            v_coords = mesh.vertex_coordinates(v)
            v_vector = Vector(*v_coords)
            vert_to_cent = centroid_vector - v_vector
            vert_to_cent *= ratio
            new_vertex = v_vector + vert_to_cent + normal_vector
            x, y, z = new_vertex
            new_verts.append(subd.add_vertex(x=x, y=y, z=z))
        
        for i,v in enumerate(face_verts):
            next_v = face_verts[(i+1) % len(face_verts)]
            new_v = new_verts[i]
            next_new_v = new_verts[(i+1) % len(face_verts)]
            new_face_key = subd.add_face([v, next_v, next_new_v, new_v])
            subd.set_face_attribute(new_face_key, 'material', 'frame')
        
        top_face_key = subd.add_face(new_verts)
        subd.set_face_attribute(top_face_key, 'material', 'glass')
        del subd.face[fkey]
    return subd

In [22]:
tapered_mesh = mesh_subdivide_tapered(mesh3, height=0.2, ratio=0.3)
draw_compas_mesh(tapered_mesh)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [23]:
tapered_mesh.get_faces_attribute(tapered_mesh.faces(), 'material')

['frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',
 'frame',
 'frame',
 'frame',
 'glass',


In [24]:
from utilities import export_obj_by_attribute

In [25]:
export_obj_by_attribute('spacestation.obj', tapered_mesh, 'material')